In [110]:
# import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
from __future__ import division
import missingno as msno
import xgboost
import numpy as np
import xgboost
from sklearn.model_selection import StratifiedKFold, GridSearchCV
# import lightgbm
import gc

[Create Features](#createFeatures)

[Merging DataFrames](#mergeDfs)

[Start Training](#startTraining)

[Renaming Columns](#renamingColumns)

In [111]:
from imblearn.over_sampling import SMOTE,RandomOverSampler

In [112]:
from utils import MyUtils

In [113]:
pd.set_option('display.max_colwidth', -1)

In [114]:
pd.set_option('display.max_columns', 50)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [115]:
test = pd.read_csv('application_test.csv')

In [116]:
print ('Reading train ...')
train = pd.read_csv('application_train.csv')
train.name = "train"
print ('Shape of train is %s'%(str(train.shape)))
bureau_balance = pd.read_csv('bureau_balance.csv')
bureau_balance.name = 'bureau_balance'
print ('The shape of bureau_balance is %s'%(str(bureau_balance.shape)))
credit_card_balance = pd.read_csv('credit_card_balance.csv')
credit_card_balance.name = 'credit_card_balance'
print ('The shape of creditcard_balance is %s'%(str(credit_card_balance.shape)))
installments_payments = pd.read_csv('installments_payments.csv')
installments_payments.name = 'installments_payments'
print ('The shape of installments_payments is %s'%(str(installments_payments.shape)))
pos_cash_balance = pd.read_csv('POS_CASH_balance.csv')
pos_cash_balance.name = 'pos_cash_balance'
print ('The shape of pos_cash_balance is %s'%(str(pos_cash_balance.shape)))
previous_application = pd.read_csv('previous_application.csv')
previous_application.name = 'previous_application'
print ('The shape of previous_application is %s'%(str(previous_application.shape)))
descriptions = pd.read_csv('descriptions.csv', encoding='ISO-8859-1')
print ('The shape of descriptions is %s'%(str(descriptions.shape)))
bureau = pd.read_csv('bureau.csv')
bureau.name = 'bureau'
print ('The shape of bureau is %s'%(str(bureau.shape)))
print ('Done!')

Reading train ...
Shape of train is (307511, 122)
The shape of bureau_balance is (27299925, 3)
The shape of creditcard_balance is (3840312, 23)
The shape of installments_payments is (13605401, 8)
The shape of pos_cash_balance is (10001358, 8)
The shape of previous_application is (1670214, 37)
The shape of descriptions is (219, 5)
The shape of bureau is (1716428, 17)
Done!


In [81]:
dataframes = [credit_card_balance, installments_payments, pos_cash_balance, previous_application, bureau]

In [82]:
for df in dataframes:        
        print("The value counts for dataframe %s is %s"%(df.name, df.SK_ID_CURR.value_counts().shape[0]))

The value counts for dataframe credit_card_balance is 103558
The value counts for dataframe installments_payments is 339587
The value counts for dataframe pos_cash_balance is 337252
The value counts for dataframe previous_application is 338857
The value counts for dataframe bureau is 305811


In [86]:
test.shape[0] + train.shape[0]

356255

In [117]:
CCB_FULL = credit_card_balance

In [139]:
CCB = CCB_FULL[0:]
CCB.loc[:,'AMT_CREDIT_LIMIT_ACTUAL1'] = CCB.loc[:,'AMT_CREDIT_LIMIT_ACTUAL']
df_2 = df[df['B'] == 'b.2'].copy()

/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [102]:
grp = CCB.groupby(by=['SK_ID_CURR'])['SK_ID_PREV'].nunique().reset_index().rename(index = str, columns = {'SK_ID_PREV': 'NO_LOANS'})

In [126]:
def f(x1, x2):
    balance = x1.max()
    limit = x2.max()
    return (balance/limit)

In [131]:
grp = CCB.groupby(by=['SK_ID_CURR', 'SK_ID_PREV', 'AMT_CREDIT_LIMIT_ACTUAL']) \
            .apply(lambda x: f(x.AMT_BALANCE, x.AMT_CREDIT_LIMIT_ACTUAL1)) \
            .reset_index() \
            .rename(index = str, columns={0: 'CREDIT_LOAD1'})

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in double_scalars
  after removing the cwd from sys.path.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in double_scalars
  after removing the cwd from sys.path.


In [134]:
grp = CCB.groupby(by=['SK_ID_CURR', 'SK_ID_PREV', 'AMT_CREDIT_LIMIT_ACTUAL']).sum()

In [135]:
grp

MONTHS_BALANCE  AMT_BALANCE  \
SK_ID_CURR SK_ID_PREV AMT_CREDIT_LIMIT_ACTUAL                                
100006     1489396    270000                  -21             0.000          
100011     1843384    90000                   -104            0.000          
                      180000                  -2745           4031676.225    
100013     2038692    45000                   -253            0.000          
                      157500                  -4403           1743352.245    
100021     2594025    675000                  -170            0.000          
100023     1499902    45000                   -38             0.000          
                      225000                  -22             0.000          
100028     1914954    225000                  -1225           396167.850     
100036     2621538    45000                   -36             0.000          
                      90000                   -54             0.000          
100042     2137382    45000                   -231            0.000          
                      90000                   -3339           2801919.375    
100043     1557583    22500                   -3              0.000          
                      45000                   -75             0.000          
                      427500                  -483            6882895.800    
100047     1472630    45000                   -99             0.000          
                      450000                  -675            0.000          
100048     1060558    225000                  -496            1854127.260    
100049     1209367    90000                   -95             273268.575     
                      135000                  -9              353114.280     
100050     1794209    180000                  -28             0.000          
100059     1787234    0                       -20             0.000          
                      90000                   -45             0.000          
100066     1852787    9000                    -21             0.000          
                      45000                   -29             0.000          
                      135000                  -70             0.000          
100067     2833536    45000                   -3828           2364897.465    
100075     1596605    225000                  -44             1308696.435    
100077     1691588    0                       -9              0.000          
...                                            ..                     ...    
456215     1327087    0                       -2553           0.000          
                      135000                  -2100           518065.740     
456217     2287911    180000                  -4560           15701376.060   
456219     2051360    45000                   -10             17950.500      
                      90000                   -30             79750.080      
                      135000                  -15             631140.210     
456225     2445843    180000                  -4656           11280137.985   
456233     2458558    45000                   -34             28298.835      
                      180000                  -21             365644.215     
456235     1499583    45000                   -11             0.000          
                      90000                   -54             0.000          
456236     2789041    0                       -2077           0.000          
                      180000                  -2576           2097559.080    
456237     2556618    0                       -1              0.000          
                      225000                  -119            1096242.750    
456239     2131120    0                       -88             0.000          
                      135000                  -3105           4474300.410    
                      270000                  -899            4989236.895    
456241     1509615    45000                   -65             0.000  

In [130]:
CCB.shape

(3840312, 24)

In [105]:
CCB = CCB.merge(grp, on='SK_ID_CURR', how='left')
gc.collect()

1052

In [106]:
my_utils = MyUtils(CCB)

In [109]:
my_utils.getColDesc()

MONTHS_BALANCE [ 'Month of balance relative to application date (-1 means the freshest balance date)'
 'Month of balance relative to application date (-1 means the information to the freshest monthly snapshot, 0 means the information at application - often it will be the same as -1 as many banks are not updating the information to Credit Bureau regularly )']
AMT_BALANCE ['Balance during the month of previous credit']
AMT_CREDIT_LIMIT_ACTUAL ['Credit card limit during the month of the previous credit']
AMT_DRAWINGS_ATM_CURRENT ['Amount drawing at ATM during the month of the previous credit']
AMT_DRAWINGS_CURRENT ['Amount drawing during the month of the previous credit']
AMT_DRAWINGS_OTHER_CURRENT ['Amount of other drawings during the month of the previous credit']
AMT_DRAWINGS_POS_CURRENT ['Amount drawing or buying goods during the month of the previous credit']
AMT_INST_MIN_REGULARITY ['Minimal installment for this month of the previous credit']
AMT_PAYMENT_CURRENT ['How much did the c

In [ ]:
CCB.SK_ID_PREV.value_counts()

In [98]:
CCB.NO_LOANS.isnull().value_counts()

False    3840312
Name: NO_LOANS, dtype: int64

In [73]:
descriptions[descriptions.Row == 'SK_ID_CURR'].Table

0      application_{train|test}.csv
122    bureau.csv                  
143    POS_CASH_balance.csv        
151    credit_card_balance.csv     
174    previous_application.csv    
212    installments_payments.csv   
Name: Table, dtype: object

In [8]:
cred_sort = credit_card_balance.sort_values(by=['SK_ID_CURR', 'MONTHS_BALANCE'])

In [ ]:
cred_sort[cred_sort.groupby('SK_ID_CURR').cumcount() == 0].shape

In [ ]:
cred_month_max_df = credit_card_balance.groupby('SK_ID_CURR').MONTHS_BALANCE.min().to_frame().reset_index()

In [ ]:
credit_card_balance.sort_values(by=['SK_ID_CURR', 'MONTHS_BALANCE_CRED']).head(50)

In [ ]:
zero_amt_cred = credit_card_balance[credit_card_balance.AMT_BALANCE == 0]
zero_grp_cred_df = zero_amt_cred.groupby('SK_ID_CURR').AMT_BALANCE.count().to_frame().reset_index()
zero_grp_cred_df.columns = ['SK_ID_CURR', 'ZERO_MONTHS_COUNT']

In [ ]:
head_cred[head_cred.groupby('SK_ID_CURR').cumcount() == 0]

In [ ]:
bur_bal_max = bureau_balance.groupby('SK_ID_BUREAU').MONTHS_BALANCE.max().to_frame().reset_index()

In [ ]:
bureau_balance.head()

In [ ]:
bureau.head()

In [ ]:
bur_bal_max.shape

In [ ]:
bureau.shape

In [ ]:
bureau.groupby('SK_ID_CURR').count().shape

In [ ]:
dum_bureau_balance = pd.get_dummies(bureau_balance)

In [ ]:
dum_bureau_balance.groupby('SK_ID_BUREAU').STATUS_0.count().shape

In [ ]:
bureau.SK_ID_BUREAU.value_counts().shape

In [ ]:
bur_bal_merge = bureau.merge(bureau_balance, on='SK_ID_BUREAU', how='left')

In [ ]:
bur_bal_merge.shape

In [ ]:
bur_bal_merge.isnull().sum()

In [ ]:
bureau.corr()

In [ ]:
bureau.dtypes[bureau.dtypes == 'object'].index.values

In [ ]:
credit_card_balance.groupby(['SK_ID_CURR', 'SK_ID_PREV']).AMT_BALANCE.mean().reset_index().sort_values('SK_ID_CURR')

In [ ]:
my_utils = MyUtils(bureau)
my_utils.getColDesc()

In [ ]:
my_utils.getColDesc()

In [ ]:
dum_bureau = pd.get_dummies(bureau)

In [ ]:
my_utils = MyUtils(bureau)
my_utils.groupBySkidCount('DAYS_CREDIT').shape

In [ ]:
columns_without_spaces = []
for col in dum_bureau.columns.values:
    columns_without_spaces.append(col.replace(' ', '_'))

In [ ]:
dum_bureau.columns = columns_without_spaces

In [ ]:
dum_bureau.CREDIT_CURRENCY_currency_4.min()

In [ ]:
from __future__ import division
bureau.CREDIT_CURRENCY.value_counts()

In [ ]:
dum_bureau.groupby('SK_ID_CURR')['CREDIT_ACTIVE_Active'].mean()

In [ ]:
bureau_num_dfs = {}
for col in dum_bureau.columns:
    if col != 'object':
        meaned_df = dum_bureau.groupby('SK_ID_CURR')[str(col)].mean().to_frame().reset_index()
        bureau_num_dfs.append(col, meaned_df)

In [ ]:
dum_bureau.dtypes

In [ ]:
my_utils = MyUtils(concat_with_features)
my_utils.get_nulls()

In [ ]:
pos_cash_balance.groupby('SK_ID_CURR').SK_ID_PREV.count().shape

In [ ]:
my_utils = MyUtils(pos_cash_balance)
my_utils.getColDesc()

#### Remember 'unstack' is used to unfold categorical columns with levels

### Dont delete this, It is important

In [ ]:
# pos_cash_balance.sort_values('SK_ID_CURR').head(50)
# grouped = pos_cash_balance.groupby(['SK_ID_PREV', 'MONTHS_BALANCE']).mean()

In [ ]:
corr_train = train.corr()
corr_test = test.corr()
corr_train[corr_train > 0.70][50:]
corr_test[corr_test > 0.70][:50]
corr_train[corr_train == 0.950842]

<strong>Remember to remove one of these columns in credit_card_application as they are correlated</strong>
7. CNT_DRAWINGS_ATM_CURRENT | CNT_DRAWINGS_CURRENT columns are correlated (1.0)
8. AMT_DRAWINGS_ATM_CURRENT | AMT_DRAWINGS_CURRENT are correlated(1.0)
9. SK_DPD | SK_DPD_DEF are correlated by 1.0

<strong>These are correlated in installments_payments</strong>
1. DAYS_INSTALLMENT | DAYS_ENTRY_PAYMENT (0.99)
2. AMT_INSTALMENT | AMT_PAYMENT

<strong>In pos_cash_balance</strong>
1. CNT_INSTALMENT | CNT_INSTALMENT_FUTURE (0.87)

<strong>In train</strong>
1. AMT_CREDIT | AMT_ANNUITY (0.77)
2. AMT_CREDIT | AMT_GOODS_PRICE (0.98)
3. AMT_ANNUITY | AMT_GOODS_PRICE (0.77)
4. CNT_FAM_MEMBERS | CNT_CHILDREN
5. REGION_RATING_CLIENT | REGION_RATING_CLIENT_W_CITY (0.95)

### Let's drop correlated variables
`test = test.drop(['AMT_ANNUITY', 'AMT_GOODS_PRICE', 'CNT_CHILDREN', 'REGION_RATING_CLIENT_W_CITY'], axis=1)`

### Calculating some correlations

In [ ]:
my_utils = MyUtils(credit_card_balance)
my_utils.getDfOfMax().sort_values('SK_ID_PREV')
corr_df = my_utils.getDfOfMax()[my_utils.getDfOfMax().SK_ID_PREV == 2243411].sort_values('MONTHS_BALANCE').corr()
corr_df[corr_df == 1]

### How to get frequency distribution of numpy array

In [ ]:
##How to get frequency distribution in numpy array
y = np.bincount(credit_card_balance.SK_ID_PREV.value_counts().values)
ii = np.nonzero(y)[0]
zip(ii,y[ii])

### Checking if skids are in two dataframes

In [ ]:
train_skids = train.SK_ID_CURR
test_skids = test.SK_ID_CURR
appended = train_skids.append(test_skids)
assert len(appended) == len(train_skids) + len(test_skids)
not_in_prev = [ skid for skid in appended if skid not in  previous_application.SK_ID_CURR.unique()]
len(not_in_prev)
not_in_prev_df = pd.DataFrame(not_in_prev)
not_in_prev_df.columns = ['SK_ID_CURR']
# not_in_prev_df.SK_ID_CURR.value_counts()

# len(appended)
# previous_application.SK_ID_CURR.unique()

### Renaming colums which are similar
<a id="renamingColumns"/>
Not doing this is causing problems in xgboost. Naming is by adding dataframe name at the end

In [12]:
credit_card_balance['NAME_CONTRACT_STATUS_CRED'] = credit_card_balance.NAME_CONTRACT_STATUS
credit_card_balance = credit_card_balance.drop(['NAME_CONTRACT_STATUS'], axis=1)
credit_card_balance['MONTHS_BALANCE_CRED'] = credit_card_balance.MONTHS_BALANCE
credit_card_balance = credit_card_balance.drop(['MONTHS_BALANCE'], axis=1)

bureau_balance['MONTHS_BALANCE_BUR'] = bureau_balance.MONTHS_BALANCE
bureau_balance = bureau_balance.drop(['MONTHS_BALANCE'], axis=1)
pos_cash_balance['MONTHS_BALANCE_POS'] = pos_cash_balance.MONTHS_BALANCE
pos_cash_balance = pos_cash_balance.drop(['MONTHS_BALANCE'], axis=1)
pos_cash_balance['SK_DPD_DEF_POS'] = pos_cash_balance.SK_DPD_DEF
pos_cash_balance = pos_cash_balance.drop(['SK_DPD_DEF'], axis=1)
pos_cash_balance['NAME_CONTRACT_STATUS_POS'] = pos_cash_balance.NAME_CONTRACT_STATUS
pos_cash_balance = pos_cash_balance.drop(['NAME_CONTRACT_STATUS'], axis=1)


previous_application['HOUR_APPR_PROCESS_START_PREV'] = previous_application.HOUR_APPR_PROCESS_START
previous_application = previous_application.drop(['HOUR_APPR_PROCESS_START'], axis=1)
previous_application['NAME_TYPE_SUITE_PREV'] = previous_application.NAME_TYPE_SUITE
previous_application = previous_application.drop(['NAME_TYPE_SUITE'], axis=1)
previous_application['WEEKDAY_APPR_PROCESS_START_PREV'] = previous_application.WEEKDAY_APPR_PROCESS_START
previous_application = previous_application.drop(['WEEKDAY_APPR_PROCESS_START'], axis=1)
previous_application['NAME_CONTRACT_TYPE_PREV'] = previous_application.NAME_CONTRACT_TYPE
previous_application = previous_application.drop(['NAME_CONTRACT_TYPE'], axis=1)
previous_application['AMT_CREDIT_PREV'] = previous_application.AMT_CREDIT
previous_application = previous_application.drop(['AMT_CREDIT'], axis=1)
previous_application['AMT_GOODS_PRICE_PREV'] = previous_application.AMT_GOODS_PRICE
previous_application = previous_application.drop(['AMT_GOODS_PRICE'], axis=1)
previous_application['NAME_CONTRACT_STATUS_PREV'] = previous_application.NAME_CONTRACT_STATUS
previous_application= previous_application.drop(['NAME_CONTRACT_STATUS'], axis=1)

In [ ]:
# if 'NAME_CONTRACT_STATUS' in previous_application.columns:
#     credit_card_balance['NAME_CONTRACT_STATUS_CRED'] = credit_card_balance.NAME_CONTRACT_STATUS

<a id="createFeatures">Starting to create feature columns</a>

Columns with Levels needs to be unstacked after group by
<p><strong>This is to get pure columns. It is almost like creating dummies</strong></p>

### Bureau related features

In [13]:
feature_dataframes = {} #creating a dictionary of dataframes

In [14]:
bureau_cred_curr = bureau[['SK_ID_CURR', 'CREDIT_ACTIVE', 'CREDIT_CURRENCY']]

In [15]:
dum_bureau_cred_curr = pd.get_dummies(bureau_cred_curr)
dum_bureau_cred_curr.columns = ['SK_ID_CURR', 'credit_active', 'credit_baddebt', 'credit_closed', 'credit_sold',
                           'currency1', 'currency2', 'currency3', 'currency4']

bureau_active = dum_bureau_cred_curr.groupby('SK_ID_CURR').credit_active.sum().to_frame().reset_index()
bureau_total = bureau.groupby('SK_ID_CURR').SK_ID_BUREAU.count().to_frame().reset_index()

merge_active_burtotal = bureau_active.merge(bureau_total, on='SK_ID_CURR', how='inner')
merge_active_burtotal['active_percent'] = merge_active_burtotal.credit_active / merge_active_burtotal.SK_ID_BUREAU

active_percent_df = merge_active_burtotal[['SK_ID_CURR', 'active_percent']]
feature_dataframes['active_percent_df'] = active_percent_df
print('Shape of active_percent_df is %s'%(str(active_percent_df.shape)))

Shape of active_percent_df is (305811, 2)


In [ ]:
feature_dataframes

In [ ]:
str(sk_dpd_def_pos_df)

In [19]:
my_utils = MyUtils(pos_cash_balance)
sk_dpd_def_pos_df = my_utils.groupBySkidMean('SK_DPD_DEF_POS')
feature_dataframes['sk_dpd_def_pos_df'] = sk_dpd_def_pos_df
sk_dpd_df = my_utils.groupBySkidMean('SK_DPD')
feature_dataframes['sk_dpd_df'] = sk_dpd_df
# sk_dpd_def_pos_df.merge(sk_dpd_df, how='left', on='SK_ID_CURR').corr()

# bur_grp_df =bureau.groupby('SK_ID_CURR').CREDIT_ACTIVE.value_counts().unstack()
# bur_grp_df.columns = ['credit_active', 'credit_bad_debt', 'credit_closed', 'credit_sold']
# bur_grp_df = bur_grp_df.fillna(value=0).reset_index()

bur_curr_df = bureau.groupby('SK_ID_CURR').CREDIT_CURRENCY.value_counts().unstack()
bur_curr_df.columns = ['currency1', 'currency2', 'currency3', 'currency4']
bur_curr_df = bur_curr_df.fillna(value=0).reset_index()
feature_dataframes['bur_curr_df'] = bur_curr_df

# bur_credtype_df = bureau.groupby('SK_ID_CURR').CREDIT_TYPE.value_counts().unstack()
# bur_credtype_df.columns = ['currency1', 'currency2', 'currency3', 'currency4']
# bur_credtype_df = bur_credtype_df.fillna(value=0).reset_index()

my_utils = MyUtils(bureau)
bur_amtcredsum_df = my_utils.groupBySkidMean('AMT_CREDIT_SUM')
feature_dataframes['bur_amtcredsum_df'] = bur_amtcredsum_df



In [20]:
assert len(feature_dataframes) == 5

In [ ]:
# bur_curr_df.shape
# max_month_df.shape


In [ ]:
bureau.head()

In [ ]:
my_utils = MyUtils(bureau)
my_utils.getColDesc()

In [ ]:
bureau.corr()

In [ ]:
bureau.sort_values(['SK_ID_CURR', 'DAYS_CREDIT']).head()

In [21]:
my_utils = MyUtils(bureau)
bur_dayscredupdate_df = my_utils.groupBySkidMean('DAYS_CREDIT_UPDATE')
feature_dataframes['bur_dayscredupdate_df'] = bur_dayscredupdate_df

bur_creddayoverdue_df = my_utils.groupBySkidMean('CREDIT_DAY_OVERDUE')
feature_dataframes['bur_creddayoverdue_df'] = bur_creddayoverdue_df

bur_days_credit_max_df = bureau.groupby('SK_ID_CURR').DAYS_CREDIT.max().to_frame().reset_index().rename(columns={'DAYS_CREDIT': 'days_credit_max'})
feature_dataframes['bur_days_credit_max_df'] = bur_days_credit_max_df

bur_days_credit_min_df = bureau.groupby('SK_ID_CURR').DAYS_CREDIT.min().to_frame().reset_index().rename(columns={'DAYS_CREDIT': 'days_credit_min'})
feature_dataframes['bur_days_credit_min_df'] = bur_days_credit_min_df

bur_days_credit_mean_df = bureau.groupby('SK_ID_CURR').DAYS_CREDIT.mean().to_frame().reset_index().rename(columns={'DAYS_CREDIT': 'days_credit_mean'})
feature_dataframes['bur_days_credit_mean_df'] = bur_days_credit_mean_df

bur_amtcred_sum_df = bureau.groupby('SK_ID_CURR').AMT_CREDIT_SUM.mean().to_frame().reset_index()
feature_dataframes['bur_amtcred_sum_df'] = bur_amtcred_sum_df

bur_amt_cred_sum_overdue_df = my_utils.groupBySkidMean('AMT_CREDIT_SUM_OVERDUE')
feature_dataframes['bur_amt_cred_sum_overdue_df'] = bur_amt_cred_sum_overdue_df

bur_amt_cred_sum_limit_df = my_utils.groupBySkidMean('AMT_CREDIT_SUM_LIMIT')
feature_dataframes['bur_amt_cred_sum_limit_df'] = bur_amt_cred_sum_limit_df

bur_amt_credit_sum_debt_df = my_utils.groupBySkidMean('AMT_CREDIT_SUM_DEBT')
feature_dataframes['bur_amt_credit_sum_debt_df'] = bur_amt_credit_sum_debt_df


In [ ]:
assert len(feature_dataframes) == 14

for (key,value) in feature_dataframes.items():
    print('Shape of %s is %s'%(key, value.shape))

In [24]:
bb_cnt_df = bureau_balance.SK_ID_BUREAU.value_counts().to_frame().reset_index()
bb_cnt_df.columns = ['SK_ID_BUREAU', 'bb_cnt']

bureau_bb_cnt_df = bureau.merge(bb_cnt_df, on='SK_ID_BUREAU', how='inner')
grp_cnt_bureau_df = bureau_bb_cnt_df.groupby('SK_ID_CURR').bb_cnt.sum().to_frame().reset_index()
feature_dataframes['grp_cnt_bureau_df'] = grp_cnt_bureau_df

In [25]:
bureau_cnt_df = bureau.SK_ID_CURR.value_counts().to_frame().reset_index()
bureau_cnt_df.columns = ['SK_ID_CURR', 'bureau_cnt']

In [26]:
my_utils = MyUtils(bureau)
bureau_cnt_df = my_utils.skidCount('bureau_')
feature_dataframes['bureau_cnt_df'] = bureau_cnt_df

In [27]:
burbal_df = bureau_balance['SK_ID_BUREAU'].value_counts().to_frame().reset_index()
burbal_df.columns = ['SK_ID_BUREAU',  'bureau_cnt']
feature_dataframes['burbal_df'] = burbal_df

In [26]:
bureau[['SK_ID_CURR', 'SK_ID_BUREAU']].merge(burbal_df, on='SK_ID_BUREAU', how='left') \
            .bureau_cnt.isnull().value_counts()

True     942074
False    774354
Name: bureau_cnt, dtype: int64

### Merging Bureau and Bureau balance
Remember that each one sk_id_curr is asoociated with many sk_id_bureau which in turn has monthly records so will have to multiply both

In [28]:
#merge bureau and bureau_balance
bureau_burbal = bureau[['SK_ID_CURR', 'SK_ID_BUREAU']].merge(burbal_df, on='SK_ID_BUREAU', how='inner')
bureau_burbal.bureau_cnt = bureau_burbal.bureau_cnt.fillna(value=0)
bureau_burbal_df = bureau_burbal.groupby('SK_ID_CURR').bureau_cnt.sum().reset_index()
feature_dataframes['bureau_burbal_df'] = bureau_burbal_df

In [29]:
bureau_burbal.shape

(774354, 3)

In [ ]:
bureau_burbal_df

### previous_application related features

In [30]:
prev_app_cnt_df = previous_application.SK_ID_CURR.value_counts().to_frame().reset_index()
prev_app_cnt_df.columns = ['SK_ID_CURR', 'prev_app_cnt']
feature_dataframes['prev_app_cnt_df'] = prev_app_cnt_df

#name_contr_df
nam_contr = previous_application[['SK_ID_CURR', 'NAME_CONTRACT_TYPE_PREV']]
nam_cntr_dummied_df = nam_contr.groupby('SK_ID_CURR').NAME_CONTRACT_TYPE_PREV.value_counts().unstack()
nam_cntr_dummied_df.columns = ['cash_loans', 'consumer_loans', 'revolving_loans', 'xna']
nam_cntr_dummied_df = nam_cntr_dummied_df.fillna(value=0).reset_index()
feature_dataframes['nam_cntr_dummied_df'] = nam_cntr_dummied_df

cnt_payment_df = previous_application.groupby('SK_ID_CURR').CNT_PAYMENT.sum().to_frame().reset_index()
feature_dataframes['cnt_payment_df'] = cnt_payment_df

### Count of skidcurr in credit card balance

In [31]:
cred_cnt_df = credit_card_balance.SK_ID_CURR.value_counts().to_frame().reset_index()
cred_cnt_df.columns = ['SK_ID_CURR', 'cred_cnt']
feature_dataframes['cred_cnt_df'] = cred_cnt_df

### Max value of months in credit card balance

In [32]:
cred_month_max_df = credit_card_balance.groupby('SK_ID_CURR').MONTHS_BALANCE_CRED.min().to_frame().reset_index()
feature_dataframes['cred_month_max_df'] = cred_month_max_df

### Number of zeros in credi card balance

In [33]:
zero_amt_cred = credit_card_balance[credit_card_balance.AMT_BALANCE == 0]
zero_grp_cred_df = zero_amt_cred.groupby('SK_ID_CURR').AMT_BALANCE.count().to_frame().reset_index()
zero_grp_cred_df.columns = ['SK_ID_CURR', 'ZERO_MONTHS_COUNT']
feature_dataframes['zero_grp_cred_df'] = zero_grp_cred_df

In [34]:
my_utils = MyUtils(df=credit_card_balance)
amt_balance_df = my_utils.groupBySkidMean('AMT_BALANCE')
feature_dataframes['amt_balance_df'] = amt_balance_df

In [ ]:
credit_card_balance.corr()

In [ ]:
my_utils.getColDesc()

In [35]:
my_utils = MyUtils(previous_application)
amt_credit_df = my_utils.groupBySkidMean('AMT_CREDIT_PREV')
feature_dataframes['amt_credit_df'] = amt_credit_df

In [ ]:
previous_application.corr()

In [37]:
cols_select = ['SK_ID_CURR', 'NAME_CONTRACT_STATUS_PREV']
prevapp_select = previous_application[cols_select]

In [38]:
prev_name_cntr_status_df = prevapp_select.groupby('SK_ID_CURR').NAME_CONTRACT_STATUS_PREV.value_counts().unstack().fillna(value=0)
prev_name_cntr_status_df.columns = ['NAME_CONTRACT_STATUS_PREV_approved', 'NAME_CONTRACT_STATUS_PREV_canceled', 'NAME_CONTRACT_STATUS_PREV_refused', 'NAME_CONTRACT_STATUS_PREV_unused']
prev_name_cntr_status_df = prev_name_cntr_status_df.reset_index()
feature_dataframes['prev_name_cntr_status_df'] = prev_name_cntr_status_df

In [39]:
#This is because concatted is used in the next code block
train_x = train.drop(['TARGET'], axis=1)
concatted = pd.concat([train_x, test])

In [40]:
my_utils = MyUtils(pos_cash_balance)
# count_contract_status_df = my_utils.groupBySkidCount('NAME_CONTRACT_STATUS_POS')
pos_cash_bal_dum = pd.get_dummies(pos_cash_balance)
pos_cash_active_df = pos_cash_bal_dum.groupby('SK_ID_CURR').NAME_CONTRACT_STATUS_POS_Active \
                                .sum().to_frame().reset_index()
feature_dataframes['pos_cash_active_df'] = pos_cash_active_df

total_skids_pos = pos_cash_balance.groupby('SK_ID_CURR').SK_ID_PREV.count().to_frame().reset_index()
merge_total_active_df = pos_cash_active_df.merge(total_skids_pos, on='SK_ID_CURR', how='left')
merge_total_active_df.columns = ['SK_ID_CURR', 'ACTIVE', 'TOTAL_OF_GROUP']
merge_total_active_df['ratio_active'] = merge_total_active_df.ACTIVE / merge_total_active_df.TOTAL_OF_GROUP
merge_total_active_df = merge_total_active_df.drop(['ACTIVE', 'TOTAL_OF_GROUP'], axis=1)
feature_dataframes['merge_total_active_df'] = merge_total_active_df

##from pos_cash_balance, get the minimum value of Month and Maximum value of Month as features
min_month_df = pos_cash_balance.groupby('SK_ID_CURR').MONTHS_BALANCE_POS.min().to_frame().reset_index()
feature_dataframes['min_month_df'] = min_month_df

min_month_df = pos_cash_balance.groupby('SK_ID_CURR').MONTHS_BALANCE_POS.max().to_frame().reset_index()
feature_dataframes['min_month_df'] = min_month_df

max_dpd_df = pos_cash_balance.groupby('SK_ID_CURR').SK_DPD.max().to_frame().reset_index()
feature_dataframes['max_dpd_df'] = max_dpd_df

#Making levels of name_contract_pos into feature
cols_to_select = ['SK_ID_CURR', 'NAME_CONTRACT_STATUS_POS']
pos_select = pos_cash_balance[cols_to_select]
pos_name_contract_df = pos_select.groupby('SK_ID_CURR').NAME_CONTRACT_STATUS_POS.value_counts().unstack().fillna(value=0)
pos_name_contract_df.columns = ['pos_active', 'pos_amortized_debt', 'pos_approved', 'pos_canceled', 'pos_completed', 'pos_demand', 'pos_returned_to_store', 'pos_signed', 'pos_xna']
pos_name_contract_df = pos_name_contract_df.reset_index()
feature_dataframes['pos_name_contract_df'] = pos_name_contract_df

pos_cnt_instalmentMax_df = pos_cash_balance.groupby('SK_ID_CURR').CNT_INSTALMENT.max().to_frame().reset_index()

cred_card_grp_df = credit_card_balance.groupby('SK_ID_CURR').SK_ID_PREV.count().to_frame().reset_index()
concatted_select = concatted.iloc[:,0].to_frame() ##concatted is by joining test and train
cred_card_grp_df.columns = ['SK_ID_CURR', 'HAS_CRED']

concat_select_cred_df = concatted_select.merge(cred_card_grp_df, on='SK_ID_CURR', how='left')
concat_select_cred_df.HAS_CRED = concat_select_cred_df.HAS_CRED.apply(lambda x: 0 if x != x else 1 ) #checking if x is null
feature_dataframes['concat_select_cred_df'] = concat_select_cred_df


In [41]:
assert total_skids_pos.shape[0] == pos_cash_active_df.shape[0]

In [ ]:
pos_cash_balance.head()

In [42]:
pos_cnt_instalmentMax_df = pos_cash_balance.groupby('SK_ID_CURR').CNT_INSTALMENT.max().to_frame().reset_index()
feature_dataframes['pos_cnt_instalmentMax_df'] = pos_cnt_instalmentMax_df

pos_cnt_instalmentMin_df = pos_cash_balance.groupby('SK_ID_CURR').CNT_INSTALMENT.min().to_frame().reset_index()
feature_dataframes['pos_cnt_instalmentMin_df'] = pos_cnt_instalmentMin_df

### In installments_payments days_installment and days_entry_payment are correlated (0.99)
amt_instalment and amt_payment are correlated (0.937)

In [43]:
installments_payments['payment_ratio'] = installments_payments.AMT_PAYMENT / installments_payments.AMT_INSTALMENT
my_utils = MyUtils(installments_payments)
payment_ratio_df = my_utils.groupBySkidMean('payment_ratio')
feature_dataframes['payment_ratio_df'] = payment_ratio_df

In [ ]:
installments_payments.head()

In [ ]:
my_utils = MyUtils(installments_payments)
my_utils.getColDesc()

In [16]:
# bureau_balance.sort_values(['SK_ID_BUREAU', 'MONTHS_BALANCE']).head(50)
bureau_balance.SK_ID_BUREAU.value_counts().shape

(817395,)

In [49]:
assert bureau.SK_ID_BUREAU.shape == bureau.SK_ID_BUREAU.unique().shape

In [44]:
for (key, value) in feature_dataframes.items():
    print(key)

active_percent_df
sk_dpd_def_pos_df
sk_dpd_df
bur_curr_df
bur_amtcredsum_df
bur_dayscredupdate_df
bur_creddayoverdue_df
bur_days_credit_max_df
bur_days_credit_min_df
bur_days_credit_mean_df
bur_amtcred_sum_df
bur_amt_cred_sum_overdue_df
bur_amt_cred_sum_limit_df
bur_amt_credit_sum_debt_df
grp_cnt_bureau_df
bureau_cnt_df
burbal_df
bureau_burbal_df
prev_app_cnt_df
nam_cntr_dummied_df
cnt_payment_df
cred_cnt_df
cred_month_max_df
zero_grp_cred_df
amt_balance_df
amt_credit_df
prev_name_cntr_status_df
pos_cash_active_df
merge_total_active_df
min_month_df
max_dpd_df
pos_name_contract_df
concat_select_cred_df
pos_cnt_instalmentMax_df
pos_cnt_instalmentMin_df
payment_ratio_df


### Start the Main Process
<a id="startTraining"/>

In [45]:
train_x = train.drop(['TARGET'], axis=1)

In [46]:
assert train_x.shape[1] == test.shape[1]

In [47]:
concatted = pd.concat([train_x, test])

In [48]:
concatted.shape

(356255, 121)

### Let's remove correlated columns here

In [49]:
# concatted = concatted.drop(['AMT_ANNUITY', 'AMT_GOODS_PRICE', 'CNT_CHILDREN', 'REGION_RATING_CLIENT_W_CITY'], axis=1)

In [50]:
# assert installments_payments[installments_payments.diff_amount.isnull()].shape[0] == installments_payments[installments_payments.diff_days.isnull()].shape[0]

In [51]:
#Since they share same rows, joint them
# join_amt_days = diff_amt_mean_df.merge(diff_days_mean_df, on='SK_ID_CURR', how='inner')

In [52]:
# concat_with_features.SK_ID_CURR.value_counts()

### Doing Joins with Engineered Features
<a id="mergeDfs"/>

In [54]:
concat_with_features = concatted.merge(pos_cash_active_df, on='SK_ID_CURR', how='left') \
            .merge(amt_credit_df, on='SK_ID_CURR', how='left') \
            .merge(amt_balance_df, on='SK_ID_CURR', how='left') \
            .merge(cnt_payment_df, on='SK_ID_CURR', how='left') \
            .merge(prev_app_cnt_df, on='SK_ID_CURR', how='left') \
            .merge(cred_cnt_df, on='SK_ID_CURR', how='left') \
            .merge(bureau_burbal_df, on='SK_ID_CURR', how='left') \
            .merge(bur_curr_df, on='SK_ID_CURR', how='left') \
            .merge(bur_days_credit_max_df, on='SK_ID_CURR', how='left') \
            .merge(bur_days_credit_min_df, on='SK_ID_CURR', how='left') \
            .merge(bur_days_credit_mean_df, on='SK_ID_CURR', how='left') \
            .merge(bur_amtcredsum_df, on='SK_ID_CURR', how='left') \
            .merge(bur_dayscredupdate_df, on='SK_ID_CURR', how='left') \
            .merge(bur_creddayoverdue_df, on='SK_ID_CURR', how='left') \
            .merge(bur_amtcred_sum_df, on='SK_ID_CURR', how='left') \
            .merge(nam_cntr_dummied_df, on='SK_ID_CURR', how='left') \
            .merge(prev_name_cntr_status_df, on='SK_ID_CURR', how='left') \
            .merge(sk_dpd_def_pos_df, on='SK_ID_CURR', how='left') \
            .merge(sk_dpd_df, on='SK_ID_CURR', how='left') \
            .merge(bur_amt_cred_sum_overdue_df, on='SK_ID_CURR', how='left') \
            .merge(bur_amt_cred_sum_limit_df, on='SK_ID_CURR', how='left') \
            .merge(bur_amt_credit_sum_debt_df, on='SK_ID_CURR', how='left') \
            .merge(min_month_df, on='SK_ID_CURR', how='left')\
            .merge(max_dpd_df, on='SK_ID_CURR', how='left') \
            .merge(pos_name_contract_df, on='SK_ID_CURR', how='left') \
            .merge(pos_cnt_instalmentMax_df, on='SK_ID_CURR', how='left') \
            .merge(pos_cnt_instalmentMin_df, on='SK_ID_CURR', how='left') \
            .merge(concat_select_cred_df, on='SK_ID_CURR', how='left') \
            .merge(active_percent_df, on='SK_ID_CURR', how='left') \
            .merge(cred_month_max_df, on='SK_ID_CURR', how='left') \
            .merge(zero_grp_cred_df, on='SK_ID_CURR', how='left') \
            .merge(payment_ratio_df, on='SK_ID_CURR', how='left') 

In [55]:
for (key, value) in feature_dataframes.items():
    print('The shape of %s is %s'%(key, value.shape))

The shape of active_percent_df is (305811, 2)
The shape of sk_dpd_def_pos_df is (337252, 2)
The shape of sk_dpd_df is (337252, 2)
The shape of bur_curr_df is (305811, 5)
The shape of bur_amtcredsum_df is (305811, 2)
The shape of bur_dayscredupdate_df is (305811, 2)
The shape of bur_creddayoverdue_df is (305811, 2)
The shape of bur_days_credit_max_df is (305811, 2)
The shape of bur_days_credit_min_df is (305811, 2)
The shape of bur_days_credit_mean_df is (305811, 2)
The shape of bur_amtcred_sum_df is (305811, 2)
The shape of bur_amt_cred_sum_overdue_df is (305811, 2)
The shape of bur_amt_cred_sum_limit_df is (305811, 2)
The shape of bur_amt_credit_sum_debt_df is (305811, 2)
The shape of grp_cnt_bureau_df is (134542, 2)
The shape of bureau_cnt_df is (305811, 2)
The shape of burbal_df is (817395, 2)
The shape of bureau_burbal_df is (134542, 2)
The shape of prev_app_cnt_df is (338857, 2)
The shape of nam_cntr_dummied_df is (338857, 5)
The shape of cnt_payment_df is (338857, 2)
The shape of

<p style="color: steelblue;"> Reminder: check if there are nulls because of merge later</p>

In [ ]:
#Checking correlation between them
# bureau_total.merge(bureau_burbal_df, on='SK_ID_CURR', how='inner').corr()

In [ ]:
concat_with_features.iloc[:,129:].head()

In [55]:
concat_with_features.columns.get_loc('AMT_BALANCE')

123

In [131]:
concatted.columns[-1]

'AMT_REQ_CREDIT_BUREAU_YEAR'

In [240]:
concat_with_features.columns.get_loc('AMT_REQ_CREDIT_BUREAU_YEAR')

120

In [56]:
#fill nas after 138th column with zeros. lower cased column names are engineered columns
concat_with_features.iloc[:,123:] = concat_with_features.iloc[:,123:].fillna(value=0)

In [57]:
y_train = train.TARGET

### Load Imputer Libraries

In [58]:
from sklearn.preprocessing import Imputer
my_imputer = Imputer()
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
# clf = LogisticRegression()

In [59]:
concat_with_features_drop = concat_with_features.drop('SK_ID_CURR', axis=1)

In [60]:
dummied = pd.get_dummies(concat_with_features_drop)

In [61]:
imputed = my_imputer.fit_transform(dummied)

In [62]:
imputed_df = pd.DataFrame(imputed)
# imputed_df.columns = dummied.columns

### Split the test and train back

In [63]:
new_x_train_df = imputed_df[:train_x.shape[0]]
new_test_df = imputed_df[train_x.shape[0]:]

In [64]:
assert new_x_train_df.shape[0] == train.shape[0] 

In [65]:
assert new_test_df.shape[0] == test.shape[0]

### Initiate XGBoost Classifier

In [308]:
# from xgboost import XGBClassifier
# model = XGBClassifier()
# n_estimators = [50, 100, 150, 200]
# max_depth = [2, 4, 6, 8]
# param_grid = dict(max_depth=max_depth, n_estimators=n_estimators)
# kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=7)
# grid_search = GridSearchCV(model, param_grid, cv=kfold, n_jobs=-1)
# grid_result = grid_search.fit(new_x_train_df, y_train)
# print 'Best %s using %s'%(str(grid_result.best_score_), str(grid_result.best_params_))

In [66]:
new_x_train_df['TARGET'] = train.TARGET

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [67]:

train_zeros = new_x_train_df[new_x_train_df.TARGET == 0]
train_ones = new_x_train_df[new_x_train_df.TARGET == 1]
sampled_indices = np.random.choice(train_zeros.index, size=[11, train_ones.shape[0]], replace=False)
print ('shape of sampled indices is %s'%(str(sampled_indices.shape)))
test_dropid = new_test_df
ensembled = np.zeros([11, test.shape[0]])
preds_list = []
for i in range(11):
    print ('starting %s th sample'%(str(i)))
    new_train = pd.concat([train_zeros.loc[sampled_indices[i]], train_ones])
    train_y = new_train.TARGET
    train_x = new_train.drop(['TARGET'], axis=1)
    
    concatted = pd.concat([train_x, test_dropid])
    concatted_dummied = pd.get_dummies(concatted)
    imputed_concatted = my_imputer.fit_transform(concatted_dummied)
    imputed_concatted_df = pd.DataFrame(imputed_concatted)
    imputed_concatted_df.columns = concatted_dummied.columns
    #split them back to train and test
    train_x_after_impute = imputed_concatted_df[:train_x.shape[0]]
    test_x = imputed_concatted_df[train_x.shape[0]:]
    clf.fit(train_x_after_impute, train_y)
    preds = clf.predict_proba(test_x)
    pred_actual = preds[:,1]
    ensembled[i] = pred_actual
print ('Done!!')



shape of sampled indices is (11, 24825)
starting 0 th sample
starting 1 th sample
starting 2 th sample
starting 3 th sample
starting 4 th sample
starting 5 th sample
starting 6 th sample
starting 7 th sample
starting 8 th sample
starting 9 th sample
starting 10 th sample
Done!!


In [ ]:
dropped = new_x_train_df.drop(['TARGET'], axis=1)
x_resampled, y_resampled = SMOTE().fit_sample(dropped, y_train)
clf.fit(x_resampled, y_resampled)
predicts = clf.predict_log_proba(new_test_df)

In [ ]:
predicts = clf.predict_proba(new_test_df)

In [ ]:
y_train.shape

In [ ]:
meaned = predicts[:,0]

In [68]:
meaned = np.mean(ensembled, axis=0)
assert meaned.shape[0] == 48744

In [ ]:
clf.fit(x_resampled, y_resampled)

In [ ]:
preds = clf.predict_proba(new_test)
pred_actual = preds[:,1]

In [ ]:
assert len(not_included) == test.shape[0] - test_merged.shape[0]

In [69]:
preds_df = pd.DataFrame(meaned)
subm = pd.concat([test.SK_ID_CURR, preds_df], axis=1)
subm.columns = ['SK_ID_CURR', 'TARGET']

In [70]:
subm_csv = subm.to_csv('submission.csv', sep=',', index=False)

In [225]:
%mv submission.csv /home/pranay/Downloads/

mv: cannot move 'submission.csv' to '/home/pranay/Downloads/': No such file or directory


In [ ]:
credit_card_balance.sort_values(by=['SK_ID_CURR', 'MONTHS_BALANCE_CRED']).head()

In [ ]:
credit_card_balance.AMT_DRAWINGS_CURRENT.plot.hist()

In [ ]:
train.shape

In [ ]:
pos_cash_balance.SK_ID_CURR.value_counts()

In [ ]:
credit_card_balance.SK_ID_CURR.value_counts()

In [ ]:
previous_application.SK_ID_CURR.value_counts()

In [ ]:
for col in credit_card_balance

In [ ]:
credit_card_balance.SK_ID_CURR.dtype == 'in'

In [ ]:
assert new_train.shape[0] == train_ones.shape[0] * 2

In [ ]:
floats = less_nulls_concat.loc[:,less_nulls_concat.dtypes == 'float64']
ints = less_nulls_concat.loc[:,less_nulls_concat.dtypes == 'int64']
objectly = less_nulls_concat.loc[:,less_nulls_concat.dtypes == 'object']

In [ ]:
floats = floats.fillna(floats.mean())
ints = ints.fillna(ints.mean())

In [ ]:
assert floats.shape[0] == concatted.shape[0]

In [ ]:
get_nulls(ints)

In [ ]:
pred_actual_df = pd.DataFrame(pred_actual)

In [ ]:
subm = pd.concat([test.SK_ID_CURR, pred_actual_df], axis=1)

In [ ]:
subm.columns = ['SK_ID_CURR', 'TARGET']

In [ ]:
subm.columns

In [ ]:
subm_csv = subm.to_csv('submission.csv', sep=',', index=False)

In [ ]:
%cp sample_submission.csv /home/pranay/Downloads/

In [ ]:
sample = pd.read_csv('sample_submission.csv', sep=',')

In [ ]:
subm.shape

In [ ]:
# sample
subm.TARGET.value_counts().plot.bar()

In [ ]:
train_y.value_counts()

In [ ]:
imputed_concatted.shape

In [ ]:
get_nulls(concatted_dummied)

In [ ]:
get_nulls(ints)

In [ ]:
objectly = objectly.fillna(objectly.mode())

In [ ]:
get_nulls(objectly)

In [ ]:
objectly.EMERGENCYSTATE_MODE.fillna(objectly.EMERGENCYSTATE_MODE.mode())

In [ ]:
msno.matrix(objectly)

In [ ]:
less_nulls_concat.dtypes.value_counts()

In [ ]:
concatted.loc[:, concatted[less]

In [ ]:
assert concatted.shape[0] == train.shape[0] + test.shape[0]

In [ ]:
credit_card_balance.head()

In [ ]:
credit_card_balance[credit_card_balance.SK_ID_CURR == 186401].sort_values(by='MONTHS_BALANCE').head()

In [ ]:
count_credit = credit_card_balance.groupby('SK_ID_CURR').SK_ID_CURR.count().sort_values(ascending=False).to_frame()

In [ ]:
credit_card_balance.groupby('SK_ID_PREV').SK_ID_PREV.count().sort_values(ascending=False)

In [ ]:
count_credit = credit_card_balance.groupby('SK_ID_CURR').SK_ID_CURR.count().sort_values(ascending=False)

In [ ]:
count_credit

In [ ]:
count_credit['COUNT'] = count_credit.SK_ID_CURR

In [ ]:
count_credit['SK_ID_CURR'] = count_credit.index

In [ ]:
merged = pd.merge(train, count_credit, how='inner', on='SK_ID_CURR')

In [ ]:
merge_test = pd.merge(test, count_credit, how='inner', on='SK_ID_CURR')

In [ ]:
assert merge_test.shape[0] == test.shape[0]

In [ ]:
merged.columns

In [ ]:
merged.COUNT.isnull().value_counts()

In [ ]:
[ col for col in credit_card_balance.SK_ID_CURR.unique() if col not in train.SK_ID_CURR ]

In [ ]:
train.TARGET.value_counts()

In [ ]:
def get_description_of_columns(df):
    for col in df.columns[2:]:
        print col, descriptions[descriptions.Row == str(col)].Description.values

In [ ]:
get_description_of_columns(credit_card_balance)

In [ ]:
for col in credit_card_balance.columns[2:]:
    print col, descriptions[descriptions.Row == str(col)].Description.values

In [ ]:
descriptions.head()

In [ ]:
train.groupby(['TARGET'])['NAME_CONTRACT_TYPE'].value_counts(normalize=True)

In [ ]:
train.groupby(['TARGET'])['CODE_GENDER'].value_counts(normalize=True)

In [ ]:
train.groupby(['TARGET'])['FLAG_OWN_CAR'].value_counts(normalize=True)

In [ ]:
train.groupby(['TARGET'])['FLAG_OWN_REALTY'].value_counts(normalize=True)

In [ ]:
train.groupby(['TARGET'])['CNT_CHILDREN'].value_counts(normalize=True)

In [ ]:
msno.matrix(train[:10000])

In [ ]:
train.head()

In [ ]:
train.shape

In [ ]:
nulls = train.isnull().sum() * 100/train.shape[0]

In [ ]:
null_test = test.isnull().sum() * 100/test.shape[0]

In [ ]:
above_fifty_null_test = null_test[null_test > 50].index

In [ ]:
less_nulls_test = null_test[null_test < 50].index
less_nulls_train = null_test[null_test < 50].index

In [ ]:
msno.matrix(test[less_nulls_test])

In [ ]:
msno.matrix(train[less_nulls_train])

In [ ]:
above_fifty_null_test

In [ ]:
above_fifty_nulls = nulls[nulls > 50].index
above_fifty_nulls

In [ ]:
[ col for col in above_fifty_nulls if col not in above_fifty_null_test ]

In [ ]:
remove_nulls = train.drop(above_fifty_nulls, axis=1)

In [ ]:
assert remove_nulls.shape[1] == train.shape[1] - above_fifty_nulls.shape[0]

In [ ]:
msno.matrix(remove_nulls)

In [ ]:
msno.matrix(remove_nulls.iloc[:100, 40:60])

In [ ]:
remove_nulls.isnull().sum() * 100 /(remove_nulls.shape[0])

In [ ]:
for col in remove_nulls.columns[remove_nulls.dtypes == 'object'].values:
    print remove_nulls.groupby(['TARGET'])[str(col)].value_counts(normalize=True)

In [ ]:
remove_nulls.dtypes.value_counts()

In [ ]:
previous_application.head()

In [ ]:
for col in previous_application.columns.values:
    print col, descriptions[descriptions.Row == str(col)].Description.unique()

In [ ]:
previous_application[previous_application.columns[previous_application.dtypes == 'float64'].values].head()

In [ ]:
previous_application[previous_application.columns[previous_application.dtypes == 'float64']].isnull().sum()

In [ ]:
previous_application.SK_ID_PREV.value_counts().head()

In [ ]:
895844 * 100 / previous_application.shape[0]

In [ ]:
pos_cash_balance.head()

In [ ]:
previous_application.dtypes.value_counts()

In [ ]:
descriptions.head()

In [ ]:
descriptions[descriptions.Row == 'AMT_GOODS_PRICE']

In [ ]:
for col in remove_nulls.columns[remove_nulls.dtypes == 'float64']:
    if (col != 'SK_ID_CURR') and (col != 'TARGET'):
        print remove_nulls.groupby(['TARGET'])[str(col)].mean()

In [ ]:
#getting columns with missing values granular
ranges = [(0,30),(30,60), (60, 120)]

for i in ranges:
    msno.matrix(remove_nulls.iloc[:100, i[0]:i[1]])

In [ ]:
remove_nulls.OCCUPATION_TYPE.isnull().value_counts(normalize=True)

In [ ]:
nulls = remove_nulls.isnull().sum() * 100/train.shape[0]

In [ ]:
null_cols = nulls[nulls > 30].index.values

In [ ]:
#dtypes of null cols in remove_nulls data frame
for col in null_cols:
    print col, remove_nulls[str(col)].dtype

In [ ]:
#dtypes of missing cols in train
for col in nulls[nulls > 50].index.values:
    print col, train[str(col)].dtypes

In [ ]:
pos_cash_balance.isnull().sum()

In [ ]:
previous_application.head()

In [ ]:
previous_application.SK_ID_CURR.value_counts().head()

In [ ]:
previous_application.isnull().sum()

In [ ]:
credit_card_balance.isnull().sum()

In [ ]:
credit_card_balance.head()